In [1]:
import torch as t
import data
import mlp_model


In [22]:
model = mlp_model.MLP(bias=True)
train, test = data.identity_data(10, 16, 2)

############### Forward pass ###############

stored_outputs = {}

for i, batch in enumerate(train):
    inputs, _, _ = batch
    outputs = model(inputs.float())
    stored_outputs[i] = outputs

    
############## Backwards pass ##############

loss_fn = t.nn.MSELoss()
optimizer = t.optim.Adam(model.parameters())

for i, batch in enumerate(train):
    optimizer.zero_grad()
    
    _, targets, _ = batch
    outputs = stored_outputs[i]
    
    loss = loss_fn(targets.float(), outputs)
    
    loss.backward()
    optimizer.step()

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [128, 16]], which is output 0 of AsStridedBackward0, is at version 2; expected version 1 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).